In [1]:
import pandas as pd
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [2]:
pd.options.display.max_columns = 195

In [6]:
df=pd.read_csv("data/Speed+Dating+Data.csv", encoding='ISO-8859-1')

## Presentation du DataFrame

In [7]:
df.shape

(8378, 195)

In [8]:
df.describe(include= "all")

,iid,id,gender,idg,condtn,wave,round,position,positin1,order,partner,pid,match,int_corr,samerace,age_o,race_o,pf_o_att,pf_o_sin,pf_o_int,pf_o_fun,pf_o_amb,pf_o_sha,dec_o,attr_o,sinc_o,intel_o,fun_o,amb_o,shar_o,like_o,prob_o,met_o,age,field,field_cd,undergra,mn_sat,tuition,race,imprace,imprelig,from,zipcode,income,goal,date,go_out,career,career_c,sports,tvsports,exercise,dining,museums,art,hiking,gaming,clubbing,reading,tv,theater,movies,concerts,music,shopping,yoga,exphappy,expnum,attr1_1,sinc1_1,intel1_1,fun1_1,amb1_1,shar1_1,attr4_1,sinc4_1,intel4_1,fun4_1,amb4_1,shar4_1,attr2_1,sinc2_1,intel2_1,fun2_1,amb2_1,shar2_1,attr3_1,sinc3_1,fun3_1,intel3_1,amb3_1,attr5_1,sinc5_1,intel5_1,fun5_1,amb5_1,dec,attr,sinc,intel,fun,amb,shar,like,prob,met,match_es,attr1_s,sinc1_s,intel1_s,fun1_s,amb1_s,shar1_s,attr3_s,sinc3_s,intel3_s,fun3_s,amb3_s,satis_2,length,numdat_2,attr7_2,sinc7_2,intel7_2,fun7_2,amb7_2,shar7_2,attr1_2,sinc1_2,intel1_2,fun1_2,amb1_2,shar1_2,attr4_2,sinc4_2,intel4_2,fun4_2,amb4_2,shar4_2,attr2_2,sinc2_2,intel2_2,fun2_2,amb2_2,shar2_2,attr3_2,sinc3_2,intel3_2,fun3_2,amb3_2,attr5_2,sinc5_2,intel5_2,fun5_2,amb5_2,you_call,them_cal,date_3,numdat_3,num_in_3,attr1_3,sinc1_3,intel1_3,fun1_3,amb1_3,shar1_3,attr7_3,sinc7_3,intel7_3,fun7_3,amb7_3,shar7_3,attr4_3,sinc4_3,intel4_3,fun4_3,amb4_3,shar4_3,attr2_3,sinc2_3,intel2_3,fun2_3,amb2_3,shar2_3,attr3_3,sinc3_3,intel3_3,fun3_3,amb3_3,attr5_3,sinc5_3,intel5_3,fun5_3,amb5_3
count,8378.000000,8377.000000,8378.000000,8378.000000,8378.000000,8378.000000,8378.000000,8378.000000,6532.000000,8378.000000,8378.000000,8368.000000,8378.000000,8220.000000,8378.000000,8274.000000,8305.000000,8289.000000,8289.000000,8289.000000,8280.000000,8271.000000,8249.000000,8378.000000,8166.000000,8091.000000,8072.000000,8018.000000,7656.000000,7302.000000,8128.000000,8060.000000,7993.000000,8283.000000,8315,8296.000000,4914,3133,3583,8315.000000,8299.000000,8299.000000,8299,7314,4279,8299.000000,8281.000000,8299.000000,8289,8240.000000,8299.000000,8299.000000,8299.000000,8299.000000,8299.000000,8299.000000,8299.000000,8299.000000,8299.000000,8299.000000,8299.000000,8299.000000,8299.000000,8299.000000,8299.000000,8299.000000,8299.000000,8277.000000,1800.000000,8299.000000,8299.000000,8299.000000,8289.000000,8279.000000,8257.000000,6489.000000,6489.000000,6489.000000,6489.000000,6489.000000,6467.000000,8299.000000,8299.000000,8299.000000,8299.000000,8289.000000,8289.000000,8273.000000,8273.000000,8273.000000,8273.000000,8273.000000,4906.000000,4906.000000,4906.000000,4906.000000,4906.000000,8378.000000,8176.000000,8101.000000,8082.000000,8028.000000,7666.000000,7311.000000,8138.000000,8069.000000,8003.000000,7205.000000,4096.000000,4096.000000,4096.000000,4096.000000,4096.000000,4096.000000,4000.00000,4000.000000,4000.000000,4000.000000,4000.000000,7463.000000,7463.000000,7433.000000,1984.000000,1955.000000,1984.000000,1984.000000,1955.000000,1974.000000,7445.000000,7463.000000,7463.000000,7463.000000,7463.000000,7463.000000,5775.000000,5775.000000,5775.000000,5775.000000,5775.000000,5775.000000,5775.000000,5775.00000,5775.000000,5775.000000,5775.000000,5775.000000,7463.000000,7463.000000,7463.000000,7463.000000,7463.000000,4377.000000,4377.000000,4377.000000,4377.000000,4377.000000,3974.000000,3974.000000,3974.000000,1496.000000,668.000000,3974.000000,3974.000000,3974.000000,3974.000000,3974.000000,3974.000000,2016.000000,2016.000000,2016.000000,2016.000000,2016.000000,2016.000000,2959.000000,2959.000000,2959.000000,2959.000000,2959.000000,2959.000000,2959.000000,2959.000000,2959.000000,2959.000000,2959.000000,2016.000000,3974.000000,3974.000000,3974.000000,3974.000000,3974.000000,2016.000000,2016.000000,2016.000000,2016.000000,2016.000000
unique,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,259,NaN,241,68,115,NaN,NaN,NaN,269,409,261,NaN,NaN,NaN,367,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

- Chaque ligne represente un speed date entre un homme et un femme 
- il y a eu 21 évenements appelés "wave" entre oct 2002 et avril 2004
- pour chaque participants, 5 à 22 personnes rencontrées par wave (= round)
- 551 personnes différentes ont participé au moins une fois à ces événements. 
- a chaque évènement, il y a eu un ratio homme/femme presque toujours équilibré
- 16,4% des speed dates se sont conclus par un match


In [9]:
# Nombre de rounds par wave
rounds_per_wave = df.groupby("wave")["round"].mean().reset_index()

fig = px.bar(
    rounds_per_wave,
    x="wave",
    y="round",
    labels={"wave": "Wave", "round": "Nombre de rounds"},
    title="Nombre de rounds par wave",
    hover_data={"wave": True, "round": True}
)

fig.show()

In [11]:
# 5 à 10 rounds pour les waves avec choix de partenaires limité
df[df["condtn"]==1]["round"].unique()

array([10,  5,  9,  8,  6,  7], dtype=int64)

In [12]:
# 11 à 22 rounds pour les waves avec choix de partenaires étendu
df[df["condtn"]==2]["round"].unique()

array([16, 19, 18, 20, 21, 14, 11, 15, 22], dtype=int64)

In [13]:
df_unique= df.drop_duplicates(subset='iid').copy()
print(f'Nombre de participant unique : {df_unique.shape[0]}')

Nombre de participant unique : 551


In [14]:
df_unique["gender"].value_counts()

gender
1    277
0    274
Name: count, dtype: int64

In [15]:
# Calculer le nombre d'hommes et de femmes par wave
gender_counts = df_unique.groupby(["wave", "gender"]).size().unstack(fill_value=0).reset_index()
gender_counts.columns = ['wave', 'Femme', 'Homme']

fig = px.bar(
    gender_counts,
    x="wave",
    y=["Femme", "Homme"],
    labels={"wave": "Wave", "value": "Nombre de participants", "variable": "Genre"},
    title="Nombre de participants par wave et par genre",
    barmode='group',
    color_discrete_map={'Femme': 'red', 'Homme': 'blue'}
)

fig.update_layout(
    xaxis_title='Wave',
    yaxis_title='Nombre de participants',
    legend_title='Genre'
)

fig.show()


In [16]:
import plotly.express as px

# Map match values to alphabetic values
match_mapping = {1: 'Match', 0: 'No Match'}
df['match_alphab'] = df['match'].map(match_mapping)

# Calculate the counts of each match type
match_counts = df["match_alphab"].value_counts().reset_index()
match_counts.columns = ['Match Type', 'Count']

# Calculate the total count
total_count = match_counts['Count'].sum()

# Calculate the percentages
match_counts['Percentage'] = (match_counts['Count'] / total_count) * 100

# Create a pie chart
fig = px.pie(
    match_counts,
    values='Count',
    names='Match Type',
    title='Match Distribution',
    hover_data={'Percentage': True},
    labels={'Percentage': 'Percentage'}
)

# Update layout for better readability and adjust size
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.update_layout(
    width=600,  # Adjust the width as needed
    height=400  # Adjust the height as needed
)

# Show the plot
fig.show()


In [18]:
match_per_wave = df.groupby("wave")[["match", "round"]].mean().reset_index()
match_per_wave["match"] = match_per_wave["match"].round(2)

# Ajouter une colonne 'round_color' selon le seuil de 10
match_per_wave["round_color"] = np.where(
    match_per_wave["round"] <= 10,
    "Limited choice (≤10 round)",
    "Extensive choice (>10 round)"
)

# Barplot avec couleurs catégorielles
fig = px.bar(
    match_per_wave,
    x="wave",
    y="match",
    color="round_color",  # Utilisation de la catégorie
    color_discrete_map={
        "Limited choice (≤10 round)": "lightgreen",
        "Extensive choice (>10 round)": "darkgreen"
    },
    labels={
        "wave": "Wave",
        "match": "Taux de match",
        "round_color": "Nombre de rounds"
    },
    title="Taux de match par wave selon le nombre de round",
    hover_data={"wave": True, "match": True, "round": True}
)

fig.show()

Le nombre de round ne semble pas influencer le taux de match

In [19]:
# première wave : 10 speed dates pour 20 participants
# les hommes changent de position à chaque round. les femmes restent à la même place toute la soirée
# les femmes ont un idg impaire, les hommes un idg pair
# la femme id=01 a rencontré l'homme id=01 en 4ème date de la soirée

## Missing values analysis

In [20]:
columns_with_na = df.columns[(df.isna().sum() / len(df)).round(3) != 0]
print(f"{df[columns_with_na].shape[1]} colonnes contiennent des valeurs manquantes, soit {((df[columns_with_na].shape[1])/len(df.columns))*100:.2f}% des colonnes")


181 colonnes contiennent des valeurs manquantes, soit 92.35% des colonnes


In [21]:
# Calculer le taux de valeurs manquantes par colonne
percentage_na = (df.isna().sum() / len(df)).round(3)

# Filtrer les colonnes sans valeurs manquantes
columns_without_na = percentage_na[percentage_na == 0]

# Créer un DataFrame avec les résultats
result_df_without_NA = pd.DataFrame({
    'Column Name': columns_without_na.index,
    'NA Rate': columns_without_na.values
})

# Afficher le DataFrame résultant
print("colonnes sans aucun NAN:")
print(result_df_without_NA)

colonnes sans aucun NAN:
     Column Name  NA Rate
0            iid      0.0
1             id      0.0
2         gender      0.0
3            idg      0.0
4         condtn      0.0
5           wave      0.0
6          round      0.0
7       position      0.0
8          order      0.0
9        partner      0.0
10         match      0.0
11      samerace      0.0
12         dec_o      0.0
13           dec      0.0
14  match_alphab      0.0


Les colonnes sans valeurs manquantes correspondent presque exclusivement aux informations **indispensables** au déroulement de la rencontre :
- identifiants des participants (ids, genre, id du partenaire, interlocuteurs sont issues de la même race)
- infos sur l'évènement (wave, nombre de round, position du participant, ordre de passage, les participants vont ils rencontrés plus ou moins de 10 partenaires potentiels dans la soirée)
- info sur l'outcome du date (match, decision du participant, décision du partenaire)



In [22]:
# Calculer le taux de valeurs manquantes par colonne
percentage_na = (df.isna().sum() / len(df)).round(3)

# Filtrer les colonnes avec valeur manquantes
columns_with_na = percentage_na[percentage_na != 0]

# Créer un DataFrame avec les résultats
result_df_with_NA = pd.DataFrame({
    'Column Name': columns_with_na.index,
    'NA Rate': columns_with_na.values
})

# Trier le DataFrame par 'NA Rate'
result_df_with_NA_sorted = result_df_with_NA.sort_values(by='NA Rate', ascending=False)

# Afficher le DataFrame résultant trié
print("Colonnes avec NAN (triées par taux de NAN) :")
print(result_df_with_NA_sorted)

Colonnes avec NAN (triées par taux de NAN) :
    Column Name  NA Rate
146    num_in_3    0.920
145    numdat_3    0.821
55       expnum    0.785
112      amb7_2    0.767
109     sinc7_2    0.767
..          ...      ...
38     tvsports    0.009
37       sports    0.009
21        field    0.008
26         race    0.008
1           pid    0.001

[181 rows x 2 columns]


Si beacuoup de colonnes contiennent des valeurs manquantes, celles le moins souvent remplies correspondent :
- **aux éventuelles suites des dates** (il y a t il eu d'autres dates après à l'évènment, qui a pris l'initiative de l'appel), 
- **au potentiel autoestimé du participant** (combien de personnes intéressé par son profil),
- **aux évaluations** à J+1 (Time2) ou à +3 ou 4 semaines (Time3) de l'évènement des partenaires rencontrés ou de soit même ou des caractéristiques supposées importantes par le genre oposé
- **aux resultats scolaires / niveau d'étude / revenus** 

In [23]:
#22% de valeurs manquantes dans la variable positin1. En conclure que les règles de roulement sont différentes sur les 5 premières waves
print(df["positin1"].isna().sum()/len(df['positin1'])*100,"% de remplissage")
df[df["positin1"].notnull()]["wave"].value_counts().sort_index()

22.033898305084744 % de remplissage


wave
6      50
7     512
8     200
9     800
10    162
11    882
12    392
13    180
14    720
15    684
16     96
17    280
18     72
19    450
20     84
21    968
Name: count, dtype: int64

# Participant profil analysis

In [24]:
df_unique.head()

,iid,id,gender,idg,condtn,wave,round,position,positin1,order,partner,pid,match,int_corr,samerace,age_o,race_o,pf_o_att,pf_o_sin,pf_o_int,pf_o_fun,pf_o_amb,pf_o_sha,dec_o,attr_o,sinc_o,intel_o,fun_o,amb_o,shar_o,like_o,prob_o,met_o,age,field,field_cd,undergra,mn_sat,tuition,race,imprace,imprelig,from,zipcode,income,goal,date,go_out,career,career_c,sports,tvsports,exercise,dining,museums,art,hiking,gaming,clubbing,reading,tv,theater,movies,concerts,music,shopping,yoga,exphappy,expnum,attr1_1,sinc1_1,intel1_1,fun1_1,amb1_1,shar1_1,attr4_1,sinc4_1,intel4_1,fun4_1,amb4_1,shar4_1,attr2_1,sinc2_1,intel2_1,fun2_1,amb2_1,shar2_1,attr3_1,sinc3_1,fun3_1,intel3_1,amb3_1,attr5_1,sinc5_1,intel5_1,fun5_1,amb5_1,dec,attr,sinc,intel,fun,amb,shar,like,prob,met,match_es,attr1_s,sinc1_s,intel1_s,fun1_s,amb1_s,shar1_s,attr3_s,sinc3_s,intel3_s,fun3_s,amb3_s,satis_2,length,numdat_2,attr7_2,sinc7_2,intel7_2,fun7_2,amb7_2,shar7_2,attr1_2,sinc1_2,intel1_2,fun1_2,amb1_2,shar1_2,attr4_2,sinc4_2,intel4_2,fun4_2,amb4_2,shar4_2,attr2_2,sinc2_2,intel2_2,fun2_2,amb2_2,shar2_2,attr3_2,sinc3_2,intel3_2,fun3_2,amb3_2,attr5_2,sinc5_2,intel5_2,fun5_2,amb5_2,you_call,them_cal,date_3,numdat_3,num_in_3,attr1_3,sinc1_3,intel1_3,fun1_3,amb1_3,shar1_3,attr7_3,sinc7_3,intel7_3,fun7_3,amb7_3,shar7_3,attr4_3,sinc4_3,intel4_3,fun4_3,amb4_3,shar4_3,attr2_3,sinc2_3,intel2_3,fun2_3,amb2_3,shar2_3,attr3_3,sinc3_3,intel3_3,fun3_3,amb3_3,attr5_3,sinc5_3,intel5_3,fun5_3,amb5_3
0,1,1.0,0,1,1,1,10,7,NaN,4,1,11.0,0,0.14,0,27.0,2.0,35.0,20.0,20.0,20.0,0.0,5.0,0,6.0,8.0,8.0,8.0,8.0,6.0,7.0,4.0,2.0,21.0,Law,1.0,NaN,NaN,NaN,4.0,2.0,4.0,Chicago,"60,521","69,487.00",2.0,7.0,1.0,lawyer,NaN,9.0,2.0,8.0,9.0,1.0,1.0,5.0,1.0,5.0,6.0,9.0,1.0,10.0,10.0,9.0,8.0,1.0,3.0,2.0,15.0,20.0,20.0,15.0,15.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN,35.0,20.0,15.0,20.0,5.0,5.0,6.0,8.0,8.0,8.0,7.0,NaN,NaN,NaN,NaN,NaN,1,6.0,9.0,7.0,7.0,6.0,5.0,7.0,6.0,2.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,19.44,16.67,13.89,22.22,11.11,16.67,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,7.0,8.0,7.0,6.0,NaN,NaN,NaN,NaN,NaN,1.0,1.0,0.0,NaN,NaN,15.0,20.0,20.0,15.0,15.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN
10,2,2.0,0,3,1,1,10,3,NaN,10,1,11.0,0,0.29,1,27.0,2.0,35.0,20.0,20.0,20.0,0.0,5.0,0,8.0,7.0,6.0,9.0,7.0,4.0,7.0,2.0,2.0,24.0,law,1.0,NaN,NaN,NaN,2.0,2.0,5.0,Alabama,"35,223","65,929.00",1.0,5.0,1.0,law,NaN,3.0,2.0,7.0,10.0,8.0,6.0,3.0,5.0,8.0,10.0,1.0,9.0,8.0,7.0,8.0,3.0,1.0,4.0,5.0,45.0,5.0,25.0,20.0,0.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,65.0,0.0,10.0,25.0,0.0,0.0,7.0,5.0,10.0,8.0,3.0,NaN,NaN,NaN,NaN,NaN,0,5.0,7.0,8.0,4.0,6.0,3.0,6.0,4.0,2.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.92,18.92,21.62,27.03,5.41,8.11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,6.0,8.0,9.0,4.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,30.0,5.0,40.0,15.0,0.0,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,6.0,9.0,9.0,4.0,NaN,NaN,NaN,NaN,NaN
20,3,3.0,0,5,1,1,10,9,NaN,6,1,11.0,0,-0.24,1,27.0,2.0,35.0,20.0,20.0,20.0,0.0,5.0,0,7.0,8.0,6.0,5.0,8.0,4.0,5.0,2.0,1.0,25.0,Economics,2.0,NaN,NaN,NaN,2.0,8.0,4.0,Connecticut,"6,268",NaN,6.0,3.0,1.0,Economist,NaN,3.0,8.0,7.0,8.0,5.0,5.0,8.0,4.0,5.0,7.0,8.0,7.0,7.0,7.0,5.0,8.0,7.0,4.0,2.0,35.0,10.0,35.0,10.0,10.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,50.0,0.0,20.0,30.0,0.0,0.0,8.0,9.0,8.0,9.0,8.0,NaN,NaN,NaN,NaN,NaN,0,7.0,9.0,10.0,7.0,8.0,9.0,8.0,7.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30,4,4.0,0,7,1,1,10,6,NaN,3,1,11.0,0,-0.18,1,27.0,2.0,35.0,20.0,20.0,20.0,0.0,5.0,0,6.0,7.0,8.0,7.0,7.0,5.0,6.0,3.0,2.0,23.0,Law,1.

In [25]:
gender_mapping = {0: 'Femme', 1: 'Homme'}
df_unique['gender_label'] = df_unique['gender'].map(gender_mapping)

# Créer l'histogramme
fig = px.histogram(
    df_unique,
    x='age',
    color='gender_label',  # Utiliser la colonne 'gender_label' pour colorer les barres
    title="Distribution des âges des participants",
    labels={'age': 'Âge', 'gender_label': 'Genre'},
    nbins=50,
    barmode='overlay',
    color_discrete_map={'Femme': 'red', 'Homme': 'blue'}  # Définir les couleurs pour chaque genre
)

# Mettre à jour les traces pour formater les étiquettes sans décimales
for i in range(len(fig.data)):
    fig.data[i].hovertemplate = 'Âge: %{x}<br>Genre: %{customdata[0]}<br>Count: %{y}<extra></extra>'
    fig.data[i].customdata = df_unique[df_unique['gender_label'] == fig.data[i].name][['gender_label']]

# Changer le label de l'axe des ordonnées
fig.update_layout(yaxis_title="Nombre de participants")

fig.show()

In [26]:
race_mapping = {
    1: 'Black/African American',
    2: 'European/Caucasian-American',
    3: "Latino/Hispanic American",
    4: "Asian/Pacific Islander/Asian-American",
    5: 'Native American',
    6: 'Other'
}
df_unique["Participant's_race"] = df_unique['race'].map(race_mapping)

gender_mapping = {0: 'Femme', 1: 'Homme'}
df_unique['gender_label'] = df_unique['gender'].map(gender_mapping)

# Calculer le nombre d'occurrences pour chaque combinaison de race et de genre
race_gender_counts = df_unique.groupby(['Participant\'s_race', 'gender_label']).size().reset_index(name='count')

# Calculer le total par genre pour obtenir les pourcentages
total_hommes = len(df_unique[df_unique['gender_label'] == 'Homme'])
total_femmes = len(df_unique[df_unique['gender_label'] == 'Femme'])

# Ajouter une colonne de pourcentage pour chaque groupe
race_gender_counts['percentage'] = race_gender_counts.apply(
    lambda row: f"{row['count'] / total_hommes * 100:.0f}%" if row['gender_label'] == 'Homme'
    else f"{row['count'] / total_femmes * 100:.0f}%",
    axis=1
)

# Créer le graphique à barres groupées
fig = go.Figure(data=[
    go.Bar(name='Femme', x=race_gender_counts[race_gender_counts['gender_label'] == 'Femme']['Participant\'s_race'],
           y=race_gender_counts[race_gender_counts['gender_label'] == 'Femme']['count'],
           text=race_gender_counts[race_gender_counts['gender_label'] == 'Femme']['percentage'],
           textposition='outside', marker_color='indianred'),
    go.Bar(name='Homme', x=race_gender_counts[race_gender_counts['gender_label'] == 'Homme']['Participant\'s_race'],
           y=race_gender_counts[race_gender_counts['gender_label'] == 'Homme']['count'],
           text=race_gender_counts[race_gender_counts['gender_label'] == 'Homme']['percentage'],
           textposition='outside', marker_color='royalblue')
])

# Mettre à jour le layout pour un affichage clair
fig.update_layout(
    barmode='group',
    title='Répartition par race et genre',
    xaxis_title='Race',
    yaxis_title='Nombre de participants',
    width=900, 
    height=600  
)

fig.show()



In [27]:
# Mapping
race_mapping = {
    1: 'Black/African American',
    2: 'European/Caucasian-American',
    3: "Latino/Hispanic American",
    4: "Asian/Pacific Islander/Asian-American",
    5: 'Native American',
    6: 'Other'
}
df_unique["Participant's_race"] = df_unique['race'].map(race_mapping)

proportions = df_unique["Participant's_race"].value_counts(normalize=True).round(2)

race_df = proportions.reset_index()
race_df.columns = ["race", "proportion"]

# Pie chart
fig = px.pie(
    race_df,
    names="race",
    values="proportion",
    title="Répartition des participants par race",
    color_discrete_sequence=px.colors.sequential.Viridis 
)

fig.update_traces(textposition='inside', textinfo='label+percent', texttemplate='%{label}: %{percent:.0%}')
fig.show()

# Match analysis

In [28]:
# Calculer le taux de match (moyenne) par wave et race
taux_match_par_wave_genre = df.groupby(['wave', 'samerace'])['match'].mean().reset_index()

# Mapper pour lisibilité
taux_match_par_wave_genre['samerace'] = taux_match_par_wave_genre['samerace'].map({0: 'Differente race', 1: 'Same race'})

# Barplot
fig = px.bar(
    taux_match_par_wave_genre,
    x='wave',
    y='match',
    color='samerace',
    barmode='group',
    labels={'wave': 'Wave', 'match': 'Taux de match', 'samerace': 'Race'},
    title='Taux de match (match=1) par wave et par race',
    hover_data={'wave': True, 'samerace': True, 'match': ':.2f'},
    color_discrete_map={'Same race': 'blue', 'Differente race': 'red'}
)

fig.show()


In [35]:
# 1. Créer les deux sous-ensembles
df_samerace = df[df['samerace'] == 1]
df_differentrace = df[df['samerace'] == 0]

# 2. Pour chacun, compter match vs no match
match_counts_samerace = df_samerace['match_alphab'].value_counts().reset_index()
match_counts_samerace.columns = ['Match Type', 'Count']

match_counts_differentrace = df_differentrace['match_alphab'].value_counts().reset_index()
match_counts_differentrace.columns = ['Match Type', 'Count']

# 3. Créer deux pie charts dans une même figure
fig = go.Figure()

# Pie chart pour samerace = 1
fig.add_trace(go.Pie(
    labels=match_counts_samerace['Match Type'],
    values=match_counts_samerace['Count'],
    domain={'x': [0, 0.48]},  # place sur la moitié gauche
    title="Couples of same Race"
))

# Pie chart pour samerace = 0
fig.add_trace(go.Pie(
    labels=match_counts_differentrace['Match Type'],
    values=match_counts_differentrace['Count'],
    domain={'x': [0.52, 1]},  # place sur la moitié droite
    title="Couples of different Race"
))

# 4. Layout global
fig.update_layout(
    title_text="Distribution des Matchs : Couples of same Race vs Different Race",
)

# Affichage
fig.show()